In [1]:
import pandas as pd
import numpy as np

import statsmodels.api as sm
import statsmodels.formula.api as smf

from numba import jit

In [2]:
#匯入資料
AC = pd.read_csv('C:/Users/tsaiy/MyPython/Econometric/Accident.csv', 
                    encoding = 'utf-8', 
                    header = 0,       #以第一列作為column name
                 )

In [3]:
#解釋變數一覽
print(AC.columns.values)
print(len(AC))

['ID' 'Source' 'TMC' 'Severity' 'Start_Time' 'End_Time' 'Start_Lat'
 'Start_Lng' 'End_Lat' 'End_Lng' 'Distance(mi)' 'Description' 'Number'
 'Street' 'Side' 'City' 'County' 'State' 'Zipcode' 'Country' 'Timezone'
 'Airport_Code' 'Weather_Timestamp' 'Temperature(F)' 'Wind_Chill(F)'
 'Humidity(%)' 'Pressure(in)' 'Visibility(mi)' 'Wind_Direction'
 'Wind_Speed(mph)' 'Precipitation(in)' 'Weather_Condition' 'Amenity'
 'Bump' 'Crossing' 'Give_Way' 'Junction' 'No_Exit' 'Railway' 'Roundabout'
 'Station' 'Stop' 'Traffic_Calming' 'Traffic_Signal' 'Turning_Loop'
 'Sunrise_Sunset' 'Civil_Twilight' 'Nautical_Twilight'
 'Astronomical_Twilight']
536351


In [4]:
#保留變數：濕度Humidity(%) 氣溫Temperature(F) 能見度Visibility(mi) 天氣狀況Weather_Condition 地區Timezone 交通號誌Traffic_Signal

In [7]:
AC2 = AC.filter(['Severity','Humidity(%)','Temperature(F)','Visibility(mi)','Weather_Condition','Timezone','Traffic_Signal','State'])
AC2.rename(columns={"Temperature(F)": "Temperature",
                    "Visibility(mi)": "Visibility",
                    "Humidity(%)": "Humidity",
                   })

,Severity,Humidity,Temperature,Visibility,Weather_Condition,Timezone,Traffic_Signal,State
0,2,59.0,36.0,10.00,Fair,US/Eastern,False,MA
1,2,55.0,36.0,10.00,Fair,US/Eastern,False,MA
2,2,72.0,31.0,10.00,Mostly Cloudy,US/Eastern,False,MA
3,2,81.0,26.0,10.00,Fair,US/Eastern,False,MA
4,3,100.0,32.0,0.25,Fog,US/Eastern,False,MI
...,...,...,...,...,...,...,...,...
536346,2,96.0,66.0,0.10,Fog,US/Central,False,TX
536347,2,84.0,72.0,9.00,Overcast,US/Central,False,TX
536348,2,97.0,66.9,0.50,Overcast,US/Central,False,TX
536349,2,69.0,75.9,10.00,Scattered Clouds,US/Central,False,TX


In [8]:
#刪除遺漏值
AC2.dropna()
print(len(AC2))

536351


In [74]:
for i in range(len(AC2)):
    timezone = AC2.loc[i,'Timezone']

        #加入四個dummy variables：'Eastern','Central','Mountain','Pacific'
    if timezone == 'US/Eastern':
        AC2.loc[i,'Eastern'] = 1
    else:
        AC2.loc[i,'Eastern'] = 0
    if timezone == 'US/Central':
        AC2.loc[i,'Central'] = 1
    else:
        AC2.loc[i,'Central'] = 0
    if timezone == 'US/Mountain':
        AC2.loc[i,'Mountain'] = 1
    else:
        AC2.loc[i,'Mountain'] = 0
    if timezone == 'US/Pacific':
        AC2.loc[i,'Pacific'] = 1
    else:
        AC2.loc[i,'Pacific'] = 0

KeyboardInterrupt: 

In [9]:
print(AC2.Timezone.unique())
print(AC2.Severity.unique())
print(AC2.Weather_Condition.unique())
print(AC2.Traffic_Signal.unique())
print(AC2.State.unique())

['US/Eastern' 'US/Central' 'US/Mountain' 'US/Pacific' nan]
[2 3 4 1]
['Fair' 'Mostly Cloudy' 'Fog' 'Partly Cloudy' nan 'Cloudy'
 'Partly Cloudy / Windy' 'Light Rain' 'Wintry Mix' 'Light Snow' 'Rain'
 'Haze' 'Snow' 'Heavy Rain' 'Mostly Cloudy / Windy' 'Patches of Fog'
 'Heavy Snow' 'Heavy Rain / Windy' 'Light Drizzle' 'Light Freezing Rain'
 'Mist' 'Light Rain / Windy' 'Showers in the Vicinity' 'Drizzle' 'T-Storm'
 'Shallow Fog' 'Cloudy / Windy' 'Light Rain Shower' 'N/A Precipitation'
 'Freezing Rain' 'Light Rain with Thunder' 'Rain / Windy' 'Thunder'
 'Light Snow / Windy' 'Snow / Windy' 'Heavy Drizzle' 'Fair / Windy'
 'Snow and Sleet' 'Sleet' 'Heavy T-Storm' 'Light Drizzle / Windy'
 'Light Freezing Rain / Windy' 'Light Sleet' 'Heavy Sleet'
 'Wintry Mix / Windy' 'Light Snow Shower' 'Smoke' 'Drizzle and Fog'
 'T-Storm / Windy' 'Funnel Cloud' 'Thunder in the Vicinity'
 'Heavy Snow / Windy' 'Light Freezing Drizzle' 'Fog / Windy'
 'Light Snow and Sleet' 'Light Snow Grains' 'Thunder / Windy'
